In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.linear_model import LogisticRegression
import re
import string
from nltk.corpus import stopwords
from textblob import TextBlob
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [14]:
df = pd.read_csv("news.csv")

In [16]:
df['label'].unique()


array(['FAKE', 'REAL'], dtype=object)

In [17]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [19]:
df = df.replace(to_replace ="FAKE",value =0)
df =  df.replace(to_replace ="REAL",value =1)

In [20]:
df = df.drop(["Unnamed: 0"], axis = 1)

In [21]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [22]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [23]:
df["text"] = df["text"].apply(wordopt)

In [24]:
stop = stopwords.words('english')
df['text'] =df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,title,text,label
0,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,0
1,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,0
2,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,1
3,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,0
4,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,1


In [25]:
x = df["text"]
y = df["label"]

In [26]:
X_train, X_rem, y_train, y_rem = train_test_split(x,y, train_size=0.8)

In [27]:

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [28]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(5068,)
(5068,)
(633,)
(633,)
(634,)
(634,)


(None, None)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [31]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [32]:
y_train_predict_LR = LR.predict(xv_train)

In [33]:
y_test_predict_LR = LR.predict(xv_test)

In [34]:
from sklearn import metrics

acc = metrics.accuracy_score(y_train, y_train_predict_LR )
print("Train Accuracy : ",acc*100)

Train Accuracy :  95.67876874506709


In [35]:
from sklearn import metrics

acc = metrics.accuracy_score(y_test, y_test_predict_LR)
print("Test Accuracy : ",acc*100)

Test Accuracy :  90.69400630914826


In [36]:
print(classification_report(y_test,y_test_predict_LR))

              precision    recall  f1-score   support

           0       0.87      0.94      0.91       308
           1       0.94      0.87      0.91       326

    accuracy                           0.91       634
   macro avg       0.91      0.91      0.91       634
weighted avg       0.91      0.91      0.91       634



In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [39]:
y_train_predict_RF =RFC.predict(xv_train)

In [40]:
y_test_predict_RF = RFC.predict(xv_test)

In [41]:
from sklearn import metrics

acc = metrics.accuracy_score(y_train, y_train_predict_RF)
print("Train Accuracy : ",acc*100)

Train Accuracy :  100.0


In [42]:
from sklearn import metrics

acc = metrics.accuracy_score(y_test, y_test_predict_RF)
print("Test Accuracy : ",acc*100)

Test Accuracy :  89.74763406940063


In [43]:
print(classification_report(y_test,y_test_predict_RF))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       308
           1       0.92      0.88      0.90       326

    accuracy                           0.90       634
   macro avg       0.90      0.90      0.90       634
weighted avg       0.90      0.90      0.90       634



In [73]:
from sklearn import svm

In [74]:
random_state = np.random.RandomState(0)

In [75]:
svm = svm.SVC(kernel='linear', probability=True,random_state=random_state)

In [76]:
svm.fit(xv_train, y_train)

SVC(kernel='linear', probability=True,
    random_state=RandomState(MT19937) at 0x20765007E40)

In [77]:
y_train_predict_svm =svm.predict(xv_train)

In [78]:
y_test_predict_svm = svm.predict(xv_test)

In [79]:
from sklearn import metrics

acc = metrics.accuracy_score(y_train, y_train_predict_svm)
print("Train Accuracy : ",acc*100)

Train Accuracy :  99.23046566692976


In [80]:
from sklearn import metrics

acc = metrics.accuracy_score(y_test, y_test_predict_svm)
print("Test Accuracy : ",acc*100)

Test Accuracy :  92.74447949526814


In [81]:
print(classification_report(y_test,y_test_predict_svm))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       308
           1       0.94      0.91      0.93       326

    accuracy                           0.93       634
   macro avg       0.93      0.93      0.93       634
weighted avg       0.93      0.93      0.93       634



In [84]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "REAL News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_SVM = svm.predict(new_xv_test)
    
    pred_RFC = RFC.predict(new_xv_test)

    
    return print("\n\nLR Prediction: {} \nSvm Prediction: {} \nRFc Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_SVM[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [85]:
news = str(input())
manual_testing(news)

What it does suggest, however, is that Chaffetz is acting in a way that makes Dan Burton and Darrell Issa look like models of responsibility and bipartisanship. He didnâ€™t even have the decency to notify ranking member Elijah Cummings about something this explosive. If that doesnâ€™t trample on basic standards of fairness, I donâ€™t know what does. 


LR Prediction: Fake News 
Svm Prediction: Fake News 
RFc Prediction: Fake News
